In [38]:
import json
import urllib
import zipfile
import io
import numpy as np

# Load dataset
SW_DATASET_URL = 'https://github.com/SeijiEmery/shape-net-data/raw/master/car-vertex-data/shrinkwrap-data.json.zip'
with urllib.request.urlopen(SW_DATASET_URL) as f:
    with zipfile.ZipFile(io.BytesIO(f.read()), 'r') as zf:
        with zf.open('shrinkwrap-data.json', 'r') as f:
            sw_dataset = json.loads(f.read())
            
sw_keys = list(sw_dataset.keys())
sw_dimensions = set(map(len, sw_dataset.values()))
print("dataset members have the following lengths: %s (expected 294)"%(sw_dimensions,))

sw_keys = [ key for key in sw_dataset.keys() if len(sw_dataset[key]) == 294 ]
non_corrupt_values = [ x for x in sw_dataset.values() if len(x) == 294 ]
corrupt_values = [ x for x in sw_dataset.values() if len(x) != 294 ]
print("Has %s corrupt value(s), %s values, %s keys"%(
    len(corrupt_values), len(non_corrupt_values), len(sw_keys)))

sw_data = np.array(non_corrupt_values)
print("dataset size: %s"%(sw_data.shape,))

x_train, x_test = np.split(sw_data, [500], 0)
print("partitioning as train = %s, test = %s"%(x_train.shape, x_test.shape))

dataset members have the following lengths: {1158, 294} (expected 294)
Has 1 corrupt value(s), 601 values, 601 keys
dataset size: (601, 294)
partitioning as train = (500, 294), test = (101, 294)
